In [4]:
import pydicom
import json
import pandas as pd

In [55]:
with open('rdsr.json', 'r') as f:
    rdsr = json.load(f)

In [52]:
def get_nested_value(data, path):
    for key in path:
        if isinstance(data, list):
            data = data[int(key)]
        else:
            data = data[key]
    return data

data = rdsr
path = path
value = get_nested_value(data, path)
print(f"Value at path '{path}': {value}")


Value at path '['0040A730', 'Value', 15, '0040A730', 'Value', 5, '0040A730', 'Value', 2, '0040A300', 'Value', 0, '0040A30A', 'Value', 0]': 418.15


In [56]:
def find_path(data, target_value, path=None):
    # 初めて関数が呼び出された場合（パスが未定義）、パスを空のリストとして初期化
    if path is None:
        path = []

    # 入力データが辞書の場合の処理
    if isinstance(data, dict):
        # 辞書内の全てのキーと値に対して繰り返し
        for key, value in data.items():
            # 現在のパスにキーを追加し、新しいパスを生成
            new_path = path + [key]
            # 現在の値が目標の値と一致しているかチェック
            if value == target_value:
                # 一致している場合、新しいパスを返す
                return new_path
            # 現在の値が辞書またはリスト（すなわち、ネストされたデータ）である場合
            elif isinstance(value, (dict, list)):
                # 再帰的にその値の中を探索
                matched_path = find_path(value, target_value, new_path)
                # 目標の値が見つかった場合、そのパスを返す
                if matched_path:
                    return matched_path

    # 入力データがリストの場合の処理（辞書の場合と基本的に同じ）
    elif isinstance(data, list):
        for i, value in enumerate(data):
            new_path = path + [i]
            if value == target_value:
                return new_path
            elif isinstance(value, (dict, list)):
                matched_path = find_path(value, target_value, new_path)
                if matched_path:
                    return matched_path

    # 目標の値がデータ内に存在しない場合、Noneを返す
    return None


data = rdsr
target_value = 418.15
path = find_path(data, target_value)
print(f"Path to '{target_value}': {path}")


Path to '418.15': ['0040A730', 'Value', 15, '0040A730', 'Value', 5, '0040A730', 'Value', 2, '0040A300', 'Value', 0, '0040A30A', 'Value', 0]


In [53]:
def get_value_by_path(data, path):
    # パスの全てのキー／インデックスについて繰り返す
    for key in path:
        # データをそのキーでアクセスして更新（データを一段掘り下げる）
        data = data[key]
    # 最終的なデータを返す（パスの最後まで掘り下げた結果）
    return data

value = get_value_by_path(data, path)
print(value)  # 出力: mGy

418.15


* RDSRの必要な部分の構造は同じなので、必要な部分までのpathを自動で取得できるようにしておけば問題ないような気がする
* ただし、複数スキャンがある場合に対応できるのか不明ではある。

In [70]:
def find_path(data, target_value, path=None):
    # 初めて関数が呼び出された場合（パスが未定義）、パスを空のリストとして初期化
    if path is None:
        path = []

    # 入力データが辞書の場合の処理
    if isinstance(data, dict):
        # 辞書内の全てのキーと値に対して繰り返し
        for key, value in data.items():
            # 現在のパスにキーを追加し、新しいパスを生成
            new_path = path + [key]
            # 現在の値が目標の値と一致しているかチェック
            if value == target_value:
                # 一致している場合、新しいパスを返す
                return new_path
            # 現在の値が辞書またはリスト（すなわち、ネストされたデータ）である場合
            elif isinstance(value, (dict, list)):
                # 再帰的にその値の中を探索
                matched_path = find_path(value, target_value, new_path)
                # 目標の値が見つかった場合、そのパスを返す
                if matched_path:
                    return matched_path

    # 入力データがリストの場合の処理（辞書の場合と基本的に同じ）
    elif isinstance(data, list):
        for i, value in enumerate(data):
            new_path = path + [i]
            if value == target_value:
                return new_path
            elif isinstance(value, (dict, list)):
                matched_path = find_path(value, target_value, new_path)
                if matched_path:
                    return matched_path

    # 目標の値がデータ内に存在しない場合、Noneを返す
    return None

# 入力データと目標の値を設定
data = rdsr
target_value = 'HAS OBS CONTEXT'
# 目標の値までのパスを探索
path = find_path(data, target_value)
# パスを出力
print(f"Path to '{target_value}': {path}")


Path to 'HAS OBS CONTEXT': ['0040A730', 'Value', 1, '0040A010', 'Value', 0]


In [171]:
def find_all_paths(data, target_value):
    """
    この関数は指定した値(target_value)が与えられたデータ構造(data)の中に存在する全てのパスを見つける。
    データ構造は辞書(dict)やリスト(list)で構成されており、ネスト構造も持つことができる。
    """

    # 内部関数の定義
    def find_paths_inner(data, path):
        """
        この関数は再帰的にデータを探索し、目標の値が見つかったパスをall_pathsリストに追加する。
        """

        # データが辞書型の場合
        if isinstance(data, dict):
            for key, value in data.items():
                new_path = path + [key]  # 新しいパスを生成
                if value == target_value:  # 値が目標の値と一致するかチェック
                    all_paths.append(new_path)  # 一致する場合、新しいパスを追加
                # 値が辞書型またはリスト型（つまりネストしたデータ構造）である場合
                if isinstance(value, (dict, list)):
                    find_paths_inner(value, new_path)  # その値に対して再帰的に探索を行う

        # データがリスト型の場合
        elif isinstance(data, list):
            for i, value in enumerate(data):
                new_path = path + [i]  # 新しいパスを生成
                if value == target_value:  # 値が目標の値と一致するかチェック
                    all_paths.append(new_path)  # 一致する場合、新しいパスを追加
                # 値が辞書型またはリスト型（つまりネストしたデータ構造）である場合
                if isinstance(value, (dict, list)):
                    find_paths_inner(value, new_path)  # その値に対して再帰的に探索を行う

    all_paths = []  # 目標の値が見つかる全てのパスを格納するリストを初期化
    find_paths_inner(data, [])  # 探索を開始
    return all_paths  # 内部関数によって見つけられた全てのパスを返す



# 入力データと目標の値を設定
data = rdsr
# target_valueにrdsrのCode Valueを代入する
target_value = '113824'
# 目標の値までのパスを探索
path = find_all_paths(data, target_value)
# パスを出力
print(f"Path to '{target_value}': {path}")

Path to '113824': [['0040A730', 'Value', 13, '0040A730', 'Value', 4, '0040A730', 'Value', 0, '0040A043', 'Value', 0, '00080100', 'Value', 0], ['0040A730', 'Value', 14, '0040A730', 'Value', 4, '0040A730', 'Value', 0, '0040A043', 'Value', 0, '00080100', 'Value', 0], ['0040A730', 'Value', 15, '0040A730', 'Value', 4, '0040A730', 'Value', 0, '0040A043', 'Value', 0, '00080100', 'Value', 0]]


In [180]:
value = get_value_by_path(data, path[1][:-6])
value

{'0040A010': {'vr': 'CS', 'Value': ['CONTAINS']},
 '0040A040': {'vr': 'CS', 'Value': ['NUM']},
 '0040A043': {'vr': 'SQ',
  'Value': [{'00080100': {'vr': 'SH', 'Value': ['113824']},
    '00080102': {'vr': 'SH', 'Value': ['DCM']},
    '00080104': {'vr': 'LO', 'Value': ['Exposure Time']}}]},
 '0040A300': {'vr': 'SQ',
  'Value': [{'004008EA': {'vr': 'SQ',
     'Value': [{'00080100': {'vr': 'SH', 'Value': ['s']},
       '00080102': {'vr': 'SH', 'Value': ['UCUM']},
       '00080104': {'vr': 'LO', 'Value': ['s']}}]},
    '0040A30A': {'vr': 'DS', 'Value': [1.54]}}]}}

In [177]:
test = find_all_paths(data, 5.53)
print(test)
get_value_by_path(data, test[0])

[['0040A730', 'Value', 15, '0040A730', 'Value', 4, '0040A730', 'Value', 0, '0040A300', 'Value', 0, '0040A30A', 'Value', 0]]


5.53

In [103]:
path[-1][2] = 16

In [104]:
path[-1]

['0040A730',
 'Value',
 16,
 '0040A730',
 'Value',
 5,
 '0040A730',
 'Value',
 3,
 '0040A730',
 'Value',
 2,
 '0040A300',
 'Value',
 0,
 '004008EA',
 'Value',
 0,
 '00080104',
 'Value',
 0]